# DDoS Dataset Cleaning

In [1]:
import pandas as pd
import os
import numpy as np

In [ ]:
df1 = pd.read_csv('/mnt/sda1/ddos-data/01-12/DrDoS_DNS.csv', low_memory=True)
df2 = pd.read_csv('/mnt/sda1/ddos-data/01-12/DrDoS_LDAP.csv', low_memory=True)
df3 = pd.read_csv('/mnt/sda1/ddos-data/01-12/DrDoS_MSSQL.csv', low_memory=True)
df4 = pd.read_csv('/mnt/sda1/ddos-data/01-12/DrDoS_NTP.csv', low_memory=True)
df5 = pd.read_csv('/mnt/sda1/ddos-data/01-12/DrDoS_NetBIOS.csv', low_memory=True)
df6 = pd.read_csv('/mnt/sda1/ddos-data/01-12/DrDoS_SNMP.csv', low_memory=True)
df7 = pd.read_csv('/mnt/sda1/ddos-data/01-12/DrDoS_SSDP.csv', low_memory=True)
df8 = pd.read_csv('/mnt/sda1/ddos-data/01-12/DrDoS_UDP.csv', low_memory=True)
df9 = pd.read_csv('/mnt/sda1/ddos-data/01-12/Syn.csv', low_memory=True)
df10 = pd.read_csv('/mnt/sda1/ddos-data/01-12/TFTP.csv', low_memory=True, nrows=7000000)
df11 = pd.read_csv('/mnt/sda1/ddos-data/01-12/UDPLag.csv', low_memory=True, nrows=7000000)
df12 = pd.read_csv('/mnt/sda1/ddos-data/01-12/Portmap.csv', low_memory=True, nrows=20000000)

In [ ]:
dfs = [df1, df2, df3, df4, df5, df6, df7, df8, df9, df10, df11, df12]
for df in dfs:
    print(df.shape)
    

# Drop overfit/zero value columns

In [ ]:
drop_list = ['Unnamed: 0',' Timestamp', 'Flow ID', ' Source IP', ' Source Port', ' Destination IP', ' Destination Port']
i = 0
for df in dfs:
    dfs[i].drop(drop_list,axis=1,inplace=True)
    i += 1

# Remove duplicate values ignoring the Label column

In [ ]:
i = 0
for df in dfs:
    dfs[i].drop_duplicates(subset=df.columns.difference([' Label']),inplace=True)
    dfs[i] = dfs[i].sample(min(df.shape[0], 70000))
    i += 1

# Rename the Index using tha lebel names

In [ ]:
i = 0
for df in dfs:
    labels = []
    j = 0
    for index, row in df.iterrows():
        name = row[' Label']
        if '_' in name:
            name = name.split('_')[1]
        labels.append(str(j) + "-" + name)
        j += 1
    dfs[i]['Index'] = labels
    i += 1

In [ ]:
out = pd.concat(dfs)
out.to_csv('/mnt/sda1/ddos-data/01-12/all-cleaned.csv')

# Phase two cleaning

In [ ]:
final_df = pd.read_csv('/mnt/sda1/ddos-data/01-12/all-cleaned.csv', low_memory=True, index_col='Index')

In [ ]:
final_df.drop([' Label'],axis=1,inplace=True)
final_df['Zeros'] = 0
final_df.drop_duplicates(inplace=True)

In [ ]:
final_df.replace([np.inf, -np.inf], np.nan, inplace=True)
final_df = final_df.apply(pd.to_numeric, errors='coerce')

In [ ]:
final_df.to_csv('/mnt/sda1/ddos-data/01-12/all-cleaned.csv')